In [12]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets # to load a binary dataset for our model
from sklearn.preprocessing import StandardScaler #to scale the features
from sklearn.model_selection import train_test_split

In [13]:
# 0) preprocess data
# 1) model
# 2) loss and optimizer
# 3) training loop

In [14]:
# 0) preprocess data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

In [16]:
#scale the features- make the features have zero mean and unit variant
sc = StandardScaler()

In [17]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [18]:
#convert the numpy to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [19]:
y_train = y_train.view(y_train.shape[0], 1) #reshaping the y tensor
y_test = y_test.view(y_test.shape[0], 1)

In [21]:
# 1) model
# f = x*w + b, sigmoid at the end

class LogisticRegression(nn.Module):

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

In [23]:
#loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [25]:
# 3) training loop
num_epochs = 100
for epoch in range(num_epochs):
  #forward pass
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)
  #backward pass
  loss.backward()
  #updates
  optimizer.step()

  #zero gradients
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum()/ float(y_test.shape[0])
  print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 0.4605
epoch: 20, loss = 0.3996
epoch: 30, loss = 0.3573
epoch: 40, loss = 0.3259
epoch: 50, loss = 0.3016
epoch: 60, loss = 0.2821
epoch: 70, loss = 0.2660
epoch: 80, loss = 0.2525
epoch: 90, loss = 0.2408
epoch: 100, loss = 0.2307
accuracy = 0.9123
